In [10]:
import collections
import matplotlib.pyplot as plt; plt.rcParams.update({"font.family": "serif"})

import pyscf
import pyscf.cc
import pyscf.mcscf

# To get molecular geometries.
import openfermion as of
from openfermion import MolecularData
from openfermionpyscf import run_pyscf

import qiskit
from qiskit_aer import AerSimulator  # For MPS Simulator.

# To run on hardware.
import qiskit_ibm_runtime
from qiskit_ibm_runtime import SamplerV2 as Sampler

from adaptvqe.pools import DVG_CEO
from adaptvqe.algorithms.adapt_vqe import LinAlgAdapt

In [3]:
ibm_computer: str = "ibm_fez"

service = qiskit_ibm_runtime.QiskitRuntimeService(channel="local")
computer = service.backend()
sampler = Sampler(computer)

/Users/benjamindalfavero/.venv/adapt/lib/python3.13/site-packages/qiskit_ibm_runtime/fake_provider/backends/nighthawk/fake_nighthawk.py:76: UserWarning: Properties of fake_nighthawk are not intended to represent typical nighthawk error values.
  warnings.warn(


In [15]:
mol = pyscf.gto.Mole()
geom = of.chem.geometry_from_pubchem("H2")
basis = "sto-3g"
symmetry = "C2v"
multiplicity = 1
charge = 0
mol.build(
    atom=geom,
    basis=basis,
    symmetry=symmetry,
)

In [16]:
n_frozen = 0
active_space = range(n_frozen, mol.nao_nr())

In [17]:
# Get molecular integrals
scf = pyscf.scf.RHF(mol).run()
num_orbitals = len(active_space)
n_electrons = int(sum(scf.mo_occ[active_space]))
num_elec_a = (n_electrons + mol.spin) // 2
num_elec_b = (n_electrons - mol.spin) // 2
cas = pyscf.mcscf.CASCI(scf, num_orbitals, (num_elec_a, num_elec_b))
mo = cas.sort_mo(active_space, base=0)
hcore, nuclear_repulsion_energy = cas.get_h1cas(mo)
eri = pyscf.ao2mo.restore(1, cas.get_h2cas(mo), num_orbitals)

# Compute exact energy
exact_energy = cas.run().e_tot

converged SCF energy = -1.06610864931794
CASCI E = -1.10115033023262  E(CI) = -1.63032754115262  S^2 = 0.0000000


In [18]:
exact_energy

np.float64(-1.1011503302326182)

## Get the Ansatz circuit

In [26]:
mol_of = MolecularData(geom, basis, multiplicity, charge, description='H2')
mol_of = run_pyscf(mol_of, run_fci=True, run_ccsd=True)
print(mol_of.n_orbitals)

2


In [28]:
pool = DVG_CEO(mol_of)

my_adapt = LinAlgAdapt(
    pool=pool,
    molecule=mol_of,
    max_adapt_iter=1,
    recycle_hessian=True,
    tetris=True,
    verbose=True,
    threshold=0.1,
)

my_adapt.run()
data = my_adapt.data


linalg_adapt prepared with the following settings:
> Pool: DVG_CEO
> Molecule: H2 (interatomic distance r=0Å)
> Orbital Optimization: False
> Selection method: gradient
> Convergence criterion: total_g_norm
> Recycling Hessian: True
> Tetris: True (progressive optimization: False)
> Convergence threshold (gradient norm):  0.1
> Maximum number of iterations:  1
> candidates per iteration:  1

Initial energy: -1.0661086493179361

*** ADAPT-VQE Iteration 1 ***

Creating list of up to 6 operators ordered by gradient magnitude...

Non-Zero Gradients (tolerance E-8):
Operator 2: 0.39358116697094037
Operator 4: 0.39358116697094037
Operator 5: 0.39358116697094037
Total gradient norm: 0.5566078242249335
Operators under consideration (3):
[2, 4, 5]
Corresponding gradients (ordered by magnitude):
[np.float64(0.39358116697094037), np.float64(0.39358116697094037), np.float64(0.39358116697094037)]

Adding OVP-CEO.
Operator(s) added to ansatz: [4]

Screening operators with disjoint supports...
None 

In [30]:
circuit = pool.get_circuit(data.result.ansatz.indices, data.result.ansatz.coefficients)
print(circuit)

     ┌───┐┌───┐                                           ┌───┐  ┌──────────┐  »
q_0: ┤ X ├┤ H ├───────────────────────────────────────────┤ X ├──┤ Ry(-π/2) ├──»
     └─┬─┘├───┤┌───┐                ┌───┐                 └─┬─┘  └──────────┘  »
q_1: ──┼──┤ X ├┤ H ├────────────────┤ X ├───────────────────┼──────────────────»
       │  └─┬─┘└───┘┌──────────────┐└─┬─┘┌───────────────┐  │  ┌──────────────┐»
q_2: ──■────┼────■──┤ Ry(0.088109) ├──■──┤ Ry(-0.088109) ├──■──┤ Ry(0.088109) ├»
            │  ┌─┴─┐└──────────────┘     └───────────────┘     └──────────────┘»
q_3: ───────■──┤ X ├───────────────────────────────────────────────────────────»
               └───┘                                                           »
«     ┌──────────┐                                      ┌───┐┌──────────┐ ░ 
«q_0: ┤ Rz(-π/2) ├──────────────────────────────────────┤ X ├┤ Rz(-π/2) ├─░─
«     └──┬───┬───┘      ┌───┐                      ┌───┐└─┬─┘└──────────┘ ░ 
«q_1: ───┤ X ├──────────┤ H ├───────────